<a href="https://colab.research.google.com/github/RuiqianLi/drafts/blob/main/wav2vec2_large_960h_lv60_self_4_gram_fine_tune_real_29_Jun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install needed packages
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [2]:
# if use offline, can also get all needed file from your laptop, not from google drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
%%capture
!apt install git-lfs

In [ ]:
# check the information for avaliable GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 29 04:25:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Check how long this fine-tuning takes
import time
start = time.time()

In [ ]:
# get the fine-tune training and testing data
from datasets import load_dataset, load_metric, Audio
# "load_dataset" is a function using script to loading data, put the path to your data_loading script(should be a python script with .py format)
uob_voice_train = load_dataset("/content/drive/MyDrive/REAL_AUDIO/uob_loadingdata.py", split="train")# path/to/your/loading_data/transcript ,cache_dir=r"/content/drive/MyDrive/MrBrown"
uob_voice_test = load_dataset("/content/drive/MyDrive/REAL_AUDIO/uob_loadingdata.py", split="test")# path/to/your/loading_data/transcript ,cache_dir=r"/content/drive/MyDrive/MrBrown"

Reusing dataset uob_singlish (/root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3)
Reusing dataset uob_singlish (/root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3)


In [ ]:
# check whether the Dataset format data have been prepaired, and the information of the training and testing datasets
print(uob_voice_train)
print(uob_voice_test)

Dataset({
    features: ['id', 'audio', 'file', 'text'],
    num_rows: 43
})
Dataset({
    features: ['id', 'audio', 'file', 'text'],
    num_rows: 10
})


In [ ]:
# View specific information for each piece of data
print(uob_voice_test[0])
print(uob_voice_test[0]['id'])
print(uob_voice_train[0]['audio'])
print(uob_voice_train[0]['text'])#the corresponding correct text for each slice of audio
print(uob_voice_train[0]['file'])#path to audio file
print(type(uob_voice_train))# make sure the type of training data is "Dataset"

{'id': 'uob_11', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/test/uob_11.wav', 'array': array([0.02612305, 0.03210449, 0.03726196, ..., 0.00370789, 0.00317383,
       0.00094604], dtype=float32), 'sampling_rate': 16000}, 'file': '/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/test/uob_11.wav', 'text': 'this is Sophian calling from uob'}
uob_11
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_1.wav', 'array': array([0.0020752 , 0.00210571, 0.00146484, ..., 0.0213623 , 0.02160645,
       0.0173645 ], dtype=float32), 'sampling_rate': 16000}
this is aston calling from uob
/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_on

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
# have a quick look of formated training data
show_random_elements(uob_voice_train.remove_columns(["audio"]))

,id,file,text
0,other_32,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_32.wav,yes speaking
1,uob_8,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_8.wav,which is a priority platform in uob
2,uob_2,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_2.wav,uob oh i can provide you my employee id after that you can check with hotline
3,other_8,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_8.wav,very good
4,other_6,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_6.wav,hello mrs chan happy new year


In [ ]:
# remove these characters inside the text
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\[\]\(\)]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_remove_regex, '', batch["text"]).upper()
    return batch

In [ ]:
# use the "map" function do the remove in batch
uob_voice_train = uob_voice_train.map(remove_special_characters)
uob_voice_test = uob_voice_test.map(remove_special_characters)

Loading cached processed dataset at /root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3/cache-23b8c1e9392456de.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3/cache-1a3d1fa7bc8960a9.arrow


In [ ]:
show_random_elements(uob_voice_train.remove_columns(["audio"]))

,id,file,text
0,other_26,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_26.wav,OKAY SURE SO I SEE YOU AT FIVE OCLOCK AT CLEMENTI AT CLEMENTI BUT THE BRANCH WOULD BE CLOSE SO AND THEN THEY WILL FIX THE THE CAFE TO MEET SO WHATEVER IS IT OKAY I WILL TEXT YOU MY NUMBER SO IS THERE ANY CHANGES YOU CAN UPDATE ME AS WELL
1,uob_6,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_6.wav,BRANCH OF UOB
2,other_29,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_29.wav,UH NO NOT TOO BAD NOT TOO BAD I I GET BACK FROM MY TRIP TO VIETNAM SO ABOUT TWO WEEKS AGO THEN
3,other_19,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/other_19.wav,GIVE ME A SECOND GIVE ME A SECOND CAN MAYBE ON ONE
4,uob_3,/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_3.wav,TO THE UOB ONE ACCOUNT


In [ ]:
# extract all distinct letters of the training and test data and build our vocabulary from this set of letters.
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
# mapping function -- concatenate all transcriptions into one long transcription and then transforms the string into a set of chars
# use the "map" function extract characters inside the data in batch (batched=True->mapping function has access to all transcriptions at once)
vocab_train = uob_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=uob_voice_train.column_names)
vocab_test = uob_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=uob_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

the malaya-speech's vocab_list: (total 40 elements)

{"": 0, "a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7, "h": 8, "i": 9, "j": 10, "k": 11, "l": 12, "m": 13, "n": 14, "o": 15, "p": 16, "q": 17, "r": 18, "s": 19, "t": 20, "u": 21, "v": 22, "w": 23, "x": 24, "y": 25, "z": 26, "0": 27, "1": 28, "2": 29, "3": 30, "4": 31, "5": 32, "6": 33, "7": 34, "8": 35, "9": 36, "|": 37, "[UNK]": 38, "[PAD]": 39}

patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram
"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31

In [ ]:
# make vocab_dict base on the fine-tune training set
# note: the vocab_dict made here should be same as the vocab_dict of the pretrained model(the model we what to base on it do the fine-tune),
# so make sure your fine-tune training set's text should just contains a-z,0-9 ,no more, no less.
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict 

{' ': 0,
 "'": 1,
 'A': 2,
 'B': 3,
 'C': 4,
 'D': 5,
 'E': 6,
 'F': 7,
 'G': 8,
 'H': 9,
 'I': 10,
 'J': 11,
 'K': 12,
 'L': 13,
 'M': 14,
 'N': 15,
 'O': 16,
 'P': 17,
 'Q': 18,
 'R': 19,
 'S': 20,
 'T': 21,
 'U': 22,
 'V': 23,
 'W': 24,
 'X': 25,
 'Y': 26,
 'Z': 27}

In [ ]:
# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict[" "]
# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)
# len(vocab_dict)

In [ ]:
del vocab_dict[" "]
vocab_dict["[PAD]"] = 0
vocab_dict["[s]"] = 1
vocab_dict["[/s]"] = 2
vocab_dict["[UNK]"] = 3
vocab_dict["|"] = 4
vocab_dict["E"] = 5
vocab_dict["T"] = 6
vocab_dict["A"] = 7
vocab_dict["O"] = 8
vocab_dict["N"] = 9
vocab_dict["I"] = 10
vocab_dict["H"] = 11
vocab_dict["S"] = 12
vocab_dict["R"] = 13
vocab_dict["D"] = 14
vocab_dict["L"] = 15
vocab_dict["U"] = 16
vocab_dict["M"] = 17
vocab_dict["W"] = 18
vocab_dict["C"] = 19
vocab_dict["F"] = 20
vocab_dict["G"] = 21
vocab_dict["Y"] = 22
vocab_dict["P"] = 23
vocab_dict["B"] = 24
vocab_dict["V"] = 25
vocab_dict["K"] = 26
vocab_dict["'"] = 27
vocab_dict["X"] = 28
vocab_dict["J"] = 29
vocab_dict["Q"] = 30
vocab_dict["Z"] = 31

In [ ]:
vocab_dict

{"'": 27,
 'A': 7,
 'B': 24,
 'C': 19,
 'D': 14,
 'E': 5,
 'F': 20,
 'G': 21,
 'H': 11,
 'I': 10,
 'J': 29,
 'K': 26,
 'L': 15,
 'M': 17,
 'N': 9,
 'O': 8,
 'P': 23,
 'Q': 30,
 'R': 13,
 'S': 12,
 'T': 6,
 'U': 16,
 'V': 25,
 'W': 18,
 'X': 28,
 'Y': 22,
 'Z': 31,
 '[/s]': 2,
 '[PAD]': 0,
 '[UNK]': 3,
 '[s]': 1,
 '|': 4}

In [ ]:
len(vocab_dict)

32

In [ ]:
# # for malaya-speech fine-tune
# vocab_dict[""] = vocab_dict[" "]
# del vocab_dict[" "]
# vocab_dict["0"] = len(vocab_dict)
# vocab_dict["1"] = len(vocab_dict)
# vocab_dict["2"] = len(vocab_dict)
# vocab_dict["3"] = len(vocab_dict)
# vocab_dict["4"] = len(vocab_dict)
# vocab_dict["5"] = len(vocab_dict)
# vocab_dict["6"] = len(vocab_dict)
# vocab_dict["7"] = len(vocab_dict)
# vocab_dict["8"] = len(vocab_dict)
# vocab_dict["9"] = len(vocab_dict)
# vocab_dict["|"] = len(vocab_dict)# To make it clearer that " " has its own token class, we give it a more visible character |
# vocab_dict["[UNK]"] = len(vocab_dict)# add an "unknown" token so that the model can later deal with characters not encountered in training set
# vocab_dict["[PAD]"] = len(vocab_dict)# add a padding token that corresponds to CTC's "blank token"
# len(vocab_dict)

In [ ]:
# have a check, should be same as the malaya-speech's vocab_list
vocab_dict 

{"'": 27,
 'A': 7,
 'B': 24,
 'C': 19,
 'D': 14,
 'E': 5,
 'F': 20,
 'G': 21,
 'H': 11,
 'I': 10,
 'J': 29,
 'K': 26,
 'L': 15,
 'M': 17,
 'N': 9,
 'O': 8,
 'P': 23,
 'Q': 30,
 'R': 13,
 'S': 12,
 'T': 6,
 'U': 16,
 'V': 25,
 'W': 18,
 'X': 28,
 'Y': 22,
 'Z': 31,
 '[/s]': 2,
 '[PAD]': 0,
 '[UNK]': 3,
 '[s]': 1,
 '|': 4}

In [ ]:
# save the vocabulary as a json file.
import json
with open('vocab.json', 'w') as vocab_file: # path to store the fine-tuned model
    json.dump(vocab_dict, vocab_file)

In [ ]:
# use the json file to instantiate an object of the Wav2Vec2CTCTokenizer clas
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun"

In [ ]:
# tokenizer.push_to_hub(repo_name)

In [ ]:
# feature_size (int, defaults to 1) — The feature dimension of the extracted features.
# sampling_rate (int, defaults to 16000) — The sampling rate at which the audio files should be digitalized expressed in Hertz per second (Hz).
# padding_value (float, defaults to 0.0) — The value that is used to fill the padding values.
# do_normalize (bool, optional, defaults to True) — Whether or not to zero-mean unit-variance normalize the input. Normalizing can help to significantly improve the performance for some models, e.g., wav2vec2-lv60.
# return_attention_mask (bool, optional, defaults to False) — Whether or not call() should return attention_mask.
# Wav2Vec2 models that have set config.feat_extract_norm == "group", such as wav2vec2-base, have not been trained using attention_mask. For such models, input_values should simply be padded with 0 and no attention_mask should be passed.

from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
# Constructs a Wav2Vec2 processor which wraps a Wav2Vec2 feature extractor and a Wav2Vec2 CTC tokenizer into a single processor.
# plus: CTC(Connectionist Temporal Classification) can ensure that the speed of the speaker does not affect the recognition of STT. For the specific principle, please refer to this article:
# https://distill.pub/2017/ctc/
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
# have a check before re-sample to 16000
print(uob_voice_train[0]["file"])
print(uob_voice_train[0]["audio"])

/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_1.wav
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_1.wav', 'array': array([0.0020752 , 0.00210571, 0.00146484, ..., 0.0213623 , 0.02160645,
       0.0173645 ], dtype=float32), 'sampling_rate': 16000}


In [ ]:
# re-sample all the audios to 16000 Hz
uob_voice_train = uob_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
uob_voice_test = uob_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
# have a check after re-sampling to 16000
# the "sampling_rate" should be 16000 now
uob_voice_train[0]["audio"]

{'array': array([0.0020752 , 0.00210571, 0.00146484, ..., 0.0213623 , 0.02160645,
        0.0173645 ], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/ee76df3920bae2ca5f8c87ca6120f20a8fd63372733348870fd110a2be5a2d7e/uob_real_one/train/uob_1.wav',
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(uob_voice_train)-1)

print(uob_voice_train[rand_int]["text"])
# if use the jupyter notebook, can listen to the slice of audio as well.
ipd.Audio(data=uob_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

HI GOOD AFTERNOON IS THIS


In [ ]:
rand_int = random.randint(0, len(uob_voice_train)-1)

print("Target text:", uob_voice_train[rand_int]["text"])
print("Input array shape:", uob_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", uob_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: DURING THE THE PROJECT WELL I HAVE SOME TALENTS THIS IS ABOUT SIGNAL DEVELOPING BUT YA
Input array shape: (160000,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]#input_values
    batch["input_length"] = len(batch["input_values"])#input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
uob_voice_train = uob_voice_train.map(prepare_dataset, remove_columns=uob_voice_train.column_names)
uob_voice_test = uob_voice_test.map(prepare_dataset, remove_columns=uob_voice_test.column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3/cache-8fadc1a606cb0fb3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3/cache-b74d0fb132e70629.arrow


In [ ]:
print(uob_voice_train)

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 43
})


In [ ]:
# if the memory of the computer is not large, can set this threshold to drop some of the long auios inside the training set.

max_input_length_in_sec = 30.0
uob_voice_train = uob_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/uob_singlish/main/1.1.0/3a0585fa2969f53e70a446c0fee10707591967c80a8c7146ee631eed8fe9fec3/cache-b38a088ca65ed389.arrow


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h-lv60-self",
    # "patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram", #here put the path to the pre-trained model package
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

loading configuration file https://huggingface.co/facebook/wav2vec2-large-960h-lv60-self/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5681e9346f90f9fc4d72503284e96e6bbdc8bf5a38cafeb6ebf3791120b7570d.e0d02e2ed52b244ae1896cccc2beab5caccc2478b8b3d1131c14666c6e14cfdc
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-960h-lv60-self",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer

In [ ]:
# The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. 
# This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore. 
# Thus, we can set the requires_grad to False for all parameters of the feature extraction part.
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name, # path to store the fine-tuned model
  group_by_length=True,# group_by_length makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10, 
  gradient_checkpointing=True,
  fp16=True,
  save_steps=20,
  eval_steps=20,
  logging_steps=20,
  learning_rate=0.0002,# usually the smaller the learning_rate, the longer time will cost but the training will be better
  warmup_steps=0,
  save_total_limit=2,
  push_to_hub=True, #If do not want to upload the model checkpoints to the hub, simply set push_to_hub=False.
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=uob_voice_train,
    eval_dataset=uob_voice_test,
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun is already a clone of https://huggingface.co/RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


In [ ]:
# print(gpu_info)

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 43
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 110
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Wer
20,2.128300,1.523585,0.576389
40,1.301500,1.295635,0.493056
60,0.991800,1.308671,0.534722
80,0.849000,1.291402,0.513889
100,0.619100,1.289469,0.458333


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8
Saving model checkpoint to wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/checkpoint-20
Configuration saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/checkpoint-20/config.json
Model weights saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/checkpoint-20/pytorch_model.bin
Configuration saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/checkpoint-20/preprocessor_config.json
Configuration saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/preprocessor_config.json
Several commits (2) will be pushed upstream.
Deleting older checkpoint [wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/checkpoint-80] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/wa

TrainOutput(global_step=110, training_loss=1.1178543611006304, metrics={'train_runtime': 225.606, 'train_samples_per_second': 1.906, 'train_steps_per_second': 0.488, 'total_flos': 5.33020112343264e+16, 'train_loss': 1.1178543611006304, 'epoch': 10.0})

In [ ]:
end = time.time()
time_used = start-end
print(time_used)

-249.71801924705505


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun
Configuration saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/config.json
Model weights saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/pytorch_model.bin
Configuration saved in wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/preprocessor_config.json
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.18G [00:00<?, ?B/s]

Upload file runs/Jun29_08-01-51_03bcbd1794ad/events.out.tfevents.1656489721.03bcbd1794ad.1586.2:  49%|####9   …

To https://huggingface.co/RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun
   d261862..d93bdbc  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'uob_singlish', 'type': 'uob_singlish', 'args': 'main'}}
To https://huggingface.co/RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun
   d93bdbc..03d2c1b  main -> main



'https://huggingface.co/RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun/commit/d93bdbced3e22d349fbbe7c408ac9531658a71a0'

In [ ]:
# print(gpu_info)

In [ ]:
# # evaluation
# model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
# processor = Wav2Vec2Processor.from_pretrained(repo_name)

In [ ]:
# input_dict = processor(uob_voice_test[0]["input_values"], return_tensors="pt", padding=True)

# logits = model(input_dict.input_values.to("cuda")).logits

# pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
# uob_voice_test_transcription = load_dataset("RuiqianLi/Li_singlish", split="test")

In [ ]:
# print("Prediction:")
# print(processor.decode(pred_ids))

# print("\nReference:")
# print(uob_voice_test_transcription[0]["text"].lower())

In [ ]:
# %%capture
# !pip install pydub
# !pip install noisereduce

In [ ]:
# # convert mp3 file into wav
# from os import path
# from pydub import AudioSegment

# # files                                                                         
# src = "/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.mp3"# path to mp3 file
# dst = "/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav"# path to wav file

# # convert mp3 to wav                                                          
# sound = AudioSegment.from_mp3(src)
# sound.export(dst, format="wav")

In [ ]:
# # Import the AudioSegment class for processing audio and the 
# # split_on_silence function for separating out silent chunks.
# from pydub import AudioSegment
# from pydub.silence import split_on_silence

# # Define a function to normalize a chunk to a target amplitude.
# def match_target_amplitude(aChunk, target_dBFS):
#     ''' Normalize given audio chunk '''
#     change_in_dBFS = target_dBFS - aChunk.dBFS
#     return aChunk.apply_gain(change_in_dBFS)

# # Load your audio.
# song = AudioSegment.from_wav("/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav")# path to your audio

In [ ]:
# # check the number of channels
# song.channels

In [ ]:
# # if the number of channels is 2, we should convert it into mono first
# from pydub import AudioSegment
# sound = AudioSegment.from_wav("/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav")# path to your audio
# sound = sound.set_channels(1)
# sound.export("/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav", format="wav")# path to the output audio after converting

In [ ]:
# # check the number of channels, should be "1" now
# sound.channels

In [ ]:
# # noise reduce (not always good with noise reducing, can listen to it first and decide whether to do the noise-reduce)
# from scipy.io import wavfile
# import noisereduce as nr
# # load data
# rate, data = wavfile.read("/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav")
# # perform noise reduction
# reduced_noise = nr.reduce_noise(y=data, sr=rate)
# wavfile.write("/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav", rate, reduced_noise)

Here is the way if you want to reuse the fine-tuned model(model after fine-tune) to do the speech-to-text on any audio.

In [ ]:
# from pydub import AudioSegment

# def get_second_part_wav(main_wav_file, start_time, end_time, part_wav_file):
#     """
#     Cut audio into slices, get part of audio. Unit: second(s)
#     :param main_wav_file: (path/name.wav) original audio file
#     :param start_time: cut from time
#     :param end_time: cut to time
#     :param part_wav_file: (path/name.wav) after cut, save the audio slice as this file
#     :return:
#     """
#     start_time = start_time * 1000
#     end_time = end_time * 1000

#     sound = AudioSegment.from_wav(main_wav_file)
#     word = sound[start_time:end_time]

#     word.export(part_wav_file, format="wav")

# get_second_part_wav('/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me.wav', 0, 60, '/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me_first_60.wav')

In [ ]:
# import IPython.display as ipd
# import numpy as np

# data_1='/content/drive/MyDrive/MrBrown/Kim Huat in… The Moon Who Loved Me_first_60.wav'
# ipd.Audio(data_1, autoplay=True, rate=16000)

In [ ]:
# Import necessary library
# For managing audio file

import librosa

# Importing Pytorch
import torch

# Importing Wav2Vec
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Loading model
processor = Wav2Vec2Processor.from_pretrained("RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun")#here put the path to the fine-tuned model
model = Wav2Vec2ForCTC.from_pretrained("RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun")#here put the path to the fine-tuned model

def transcription(audio_path):
  audio, rate = librosa.load(audio_path, sr = 16000)

  # audio file is decoded on the fly
  inputs = processor(audio, sampling_rate=rate, return_tensors="pt")

  with torch.no_grad():
    logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)

  # transcribe speech
  transcription = processor.batch_decode(predicted_ids)
  transcription[0].lower()
  return transcription[0].lower()

print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_1.wav'))

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


this is ason calling from uob


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3


In [17]:
!pip show transformers

Name: transformers
Version: 4.20.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, tokenizers, packaging, filelock, pyyaml, requests, importlib-metadata, regex, huggingface-hub, tqdm
Required-by: 


In [1]:
# install needed packages
%%capture
!pip install transformers
!pip install pyctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

In [2]:
import librosa
import torch
from transformers import AutoModelForCTC, AutoProcessor

model = AutoModelForCTC.from_pretrained('patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram')#.to("cuda")
processor = AutoProcessor.from_pretrained('patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram')

def transcription(audio_path):
    audio, rate = librosa.load(audio_path, sr = 16000)
    inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    transcription = processor.batch_decode(logits.cpu().numpy()).text[0].lower()

    return transcription

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/198 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

In [3]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_2.wav'))

you be oh i can serve you my employer ideas as ted can the you the hot light


In [4]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_3.wav'))

sooner you will be one


In [31]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

model = Wav2Vec2ForCTC.from_pretrained('RuiqianLi/wav2vec2-large-960h-lv60-self-4-gram_fine-tune_real_29_Jun')#.to("cuda")
processor = Wav2Vec2ProcessorWithLM.from_pretrained('patrickvonplaten/wav2vec2-large-960h-lv60-self-4-gram')

def transcription(audio_path):
    audio, rate = librosa.load(audio_path, sr = 16000)
    inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    transcription = processor.batch_decode(logits.cpu().numpy()).text[0].lower()

    return transcription

In [32]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_1.wav'))

this is a son calling from ub


In [33]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_2.wav'))

ub oh i can provide you my employer idle that you can check with hotline


In [34]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_3.wav'))

to e u b one account


In [35]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_4.wav'))

new u u b one account


In [36]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/new/uob_5.wav'))

a new branch


In [37]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/CC_001_slices/CC_001_nr_6.wav'))

all can


In [38]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/CC_001_slices/CC_001_nr_7.wav'))

yes


In [39]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/CC_001_slices/CC_001_nr_5.wav'))

b oh i can provide you my employee idly that you can check with hotline if you like


In [25]:
import librosa
import torch

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model = Wav2Vec2ForCTC.from_pretrained('RuiqianLi/Malaya-speech_fine-tune_realcase_27_Jun')#.to("cuda")
processor = Wav2Vec2Processor.from_pretrained('RuiqianLi/Malaya-speech_fine-tune_realcase_27_Jun')


def transcription(audio_path):
  audio, rate = librosa.load(audio_path, sr = 16000)

  # audio file is decoded on the fly
  inputs = processor(audio, sampling_rate=rate, return_tensors="pt")

  with torch.no_grad():
    logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)

  # transcribe speech
  transcription = processor.batch_decode(predicted_ids)
  transcription[0].lower()
  return transcription[0].lower()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/CC_001_slices/CC_001_nr_1.wav'))

ya so we wondring letter curntly your saving accound right is it more for saving or transaction


In [ ]:
import IPython.display as ipd
import numpy as np

data_1='/content/drive/MyDrive/REAL_AUDIO/new/uob_9.wav'
ipd.Audio(data_1, autoplay=True, rate=16000)

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install jiwer

In [ ]:
!pip show jiwer

In [ ]:
from datasets import load_metric
wer_metric = load_metric(path='/content/drive/MyDrive/REAL_AUDIO/wer.py')

In [29]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

model = Wav2Vec2ForCTC.from_pretrained('RuiqianLi/Malaya-speech_fine-tune_realcase_27_Jun')#.to("cuda")
processor = Wav2Vec2ProcessorWithLM.from_pretrained('RuiqianLi/Malaya-speech_fine-tune_realcase_27_Jun')

def transcription(audio_path):
    audio, rate = librosa.load(audio_path, sr = 16000)
    inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    transcription = processor.batch_decode(logits.cpu().numpy()).text[0].lower()

    return transcription

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/247 [00:00<?, ?B/s]

In [30]:
print(transcription('/content/drive/MyDrive/REAL_AUDIO/CC_001_slices/CC_001_nr_1.wav'))

ValueError: ignored